In [4]:
from deoldify import device
from deoldify.device_id import DeviceId
device.set(device=DeviceId.GPU0)

import torch
torch.cuda.is_available()

from fastai.vision import *

In [5]:
from deoldify.visualize import *
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

from PIL import Image as PilImage

### Preprocess

In [8]:
# Resizes scraped image to a common size
# only needed to be run once after running image_scraper
# Legacy-Large: 800, Medium: 400, Small 3x: 600, Small 4x: 800
def preprocess_images(fn, sz=800):Colorizer_GANFineTune_bestmodel
    dest = data_prep_path_out
    img = PilImage.open(fn).convert('RGB') # some files are using LA format
    img = squarify(img, sz) # squarify and scale to size
    name = str(fn).split("/")[-1]
    name = name.split(".")[0]
    img.save(dest + name + ".jpg", "JPEG")
    
def squarify(im, sz: int=64):
    im = im.resize((sz,sz), resample=PilImage.BICUBIC)    
    return im

In [10]:
# example resize operation
data_prep_path_in = '/mnt/Data/datasets/au_bw/legacy_SR/' # source scraped file
data_prep_path_out = '/mnt/Data/datasets/au_bw/legacy_SR_3x/' # destination folder

il = ItemList.from_folder(data_prep_path_in)

for im in il:
    preprocess_images(im, 600)

In [15]:
# sets the root directory
path = Path('/mnt/Data/ILSVRC/Data/CLS-LOC')
path_hr = path
path_lr = path/'bw'

# for training with paired images
# paired at different folder before training, to reduce runtime operations
# generate with create_training_images_pair() method
path_lr_pair = path/'bw_pair'
path_hr_pair = path/'col_pair'

In [16]:
# create BW image
def create_training_images(fn,i):
    dest = path_lr/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = pimg.open(fn).convert('LA').convert('RGB')
    img.save(dest)

# create paired BW image
def create_training_images_pair(fn,i):
    dest = path_lr_pair/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = pimg.open(fn)#.convert('LA').convert('RGB')
    img_pair = pimg.new('RGB', (img.width, img.height * 2))
    img_pair.paste(img, (0,0))
    img_pair.paste(img, (0, img.height))
    img_pair.save(dest)

# create paired color (ground truth) image
def create_color_images_pair(fn,i):
    dest = path_hr_pair/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = pimg.open(fn)#.convert('LA').convert('RGB')
    img_pair = pimg.new('RGB', (img.width, img.height * 2))
    img_pair.paste(img, (0,0))
    img_pair.paste(img, (0, img.height))
    img_pair.save(dest)